In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from groq import Groq
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [4]:
class GroqLLM:
    def __init__(self, client, model):
        self.client = client
        self.model = model

    def __call__(self, query):
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": str(query)}]
        )
        return response.choices[0].message.content

# Initialize like this
llm = GroqLLM(client, "llama-3.3-70b-versatile")

In [14]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(web_paths=("https://www.pna.gov.ph/articles/list",))
document = loader.load()
print(document)

[Document(metadata={'source': 'https://www.pna.gov.ph/articles/list', 'title': 'News Articles | Philippine News Agency', 'description': 'List of news articles published by Philippine News Agency', 'language': 'en'}, page_content='\n\n\n\n\nNews Articles | Philippine News Agency\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nGOV PH\n\n\n\nOpen GOV.PH Menu\n\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nAbout\n\n\nSubscription\n\n\nTransparency\n\n\nContact\n\n\n\n\nSearch\n\n\n\n\n\nPerform Search\n\n\n\n\n\n\n\nHome\n\n\nAbout\n\n\nSubscription\n\n\nTransparency\n\n\nContact\n\n\n\n\nSearch\n\n\n\n\n\nPerform Search\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOpen GOV.PH Menu\n\n\n\n\n\n\n\n\n\n\nHome\n\n\nNational\n\n\nProvincial\n\n\nOpinion\n\n\nBusiness\n\n\nFeatures\n\n\nHealth\n\n\nForeign\n\n\nSports\n\n\nTravel\n\n\nEnvironment\n\n\nSciTech\n\n\n\n\n\n\n\nHome\n\n\nNational\n\n\nProvincial\n\n\nOpinion\n\n\nBusiness\n\n\nFeatures\n\n\nHealth\n\n\nForeign\n\n\nSports\n\n\nTravel\n\n\nEnvironment\

In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
document = text_splitter.split_documents(document)

In [16]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
embedding = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5")


/home/anuj/anaconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(document,embedding)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context
<context>
{context}
</context>
Question : {input}""")

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm,prompt)

In [20]:
retriever = db.as_retriever()

In [21]:
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [28]:
response = retrieval_chain.invoke({"input" : "give headlines of all news articles related to manila "})
print(response['answer'])

Based on the provided context, the headlines of news articles related to Manila are:

1. "MANILA – The southwest monsoon or “habagat” and the intertropical convergence zone (ITCZ) will bring cloudy skies with scattered rains over parts of the country, the weather bureau said on Sunday."
2. "DILG offers ‘buy 1, give 1’ scheme for police vehicles in NCR"
3. "MANILA – Department of the Interior and Local Government (DILG) Secretary Jonvic Remulla proposed “buy one, give one” scheme to Metro Manila mayors, offering to match every police vehicle they procure with an additional unit from the national government."
4. "MANILA – A total of 26 overseas Filipinos in Israel have been repatriated through the Philippine Embassy’s Voluntary Repatriation Program, amid heightened tensions between Israel and Iran."
5. "MANILA – The Philippines seeks a higher spot on the podium in the Southeast Asian Men’s V.League slated July 9 to 13 at the Candon City Arena in Ilocos Sur."

However, the last one is not